# Jane Street Puzzle - April 2025 - Sum One, Somewhere

## Problem Statement

<div class="center"><img src="images/sum-one-somewhere.png" style="background-color: white;" class="dark_img" alt="", width="700" height="600"></div>

For a fixed $p$, independently label the nodes of an infinite complete binary tree 0 with probability $p$, and 1 otherwise. For what $p$ is there exactly a 1/2 probability that there exists an infinite path down the tree that sums to at most 1 (that is, all nodes visited, with the possible exception of one, will be labeled 0). Find this value of $p$ accurate to 10 decimal places.

## Solution

Let $P_0$ be the probability of obtaining a path summing to 0 startting from the root. The probability that the root is 0 is $p$. The probability that a given subtree contains a path summing to zero is $P_0$. Therefore, the probability that none of the subtrees contain a path summing to zero is $(1 - P_0)^2$. Finally the probability of obtaining a path summing to zero only is the intersection of the probability that the root is zero and the probability that at least one subtree sums to zero. Therefore, we obtain the equation

\begin{equation}
    P_0 = p \left[ 1 - (1 - P_0)^2 \right].
\end{equation}

Solving the quadratic equation yields two solutions $P_0 = 0$ and $P_0 = 2 - \frac{1}{p}$. Therefore, a strictly positive $P_0$ only occurs is $p > \frac{1}{2}$.

Let $P_1$ be the probability that there exists a path summing to at most 1. With probability $(1 - p)$ we end up in a situation where we must have a subtree summing to zero, similar to the $P_0$ situation above. With probability $p$, we are still in a position where a one is allowed. This gives the equation

\begin{equation}
    P_1 = p \left[ 1 - (1 - P_1)^2 \right] + (1 - p)\left[ 1 - (1 - P_0)^2 \right].
\end{equation}

Solving this equation yields one real solution which is our answer.

In [1]:
from sympy import symbols, Eq, solve, Rational, N

# Define symbols
p, P0 = symbols('p P0')

# Step 1: Solve P0 = p * (1 - (1 - P0)**2)
eq1 = Eq(P0, p * (1 - (1 - P0)**2))
P0_solutions = solve(eq1, P0)

P0_solutions

[0, 2 - 1/p]

In [2]:
P0_sol = P0_solutions[1]

P1_value = Rational(1, 2)
p_solutions = []

rhs = p * (1 - (1 - P1_value)**2) + (1 - p) * (1 - (1 - P0_sol)**2)
eq2 = Eq(P1_value, rhs)
sols = solve(eq2, p)
for sol in sols:
    # Evaluate to 12 decimal places
    numeric_sol = N(sol, 10)
    p_solutions.append(numeric_sol)

# Print results
for i, ps in enumerate(p_solutions, 1):
    print(f"Solution {i}: p ≈ {ps}")

Solution 1: p ≈ 1.401364879 + 0.7409710153*I
Solution 2: p ≈ 1.401364879 - 0.7409710153*I
Solution 3: p ≈ 0.5306035754


Below is a Monte Carlo simulation to verify the result. We expect to find a probability close to 1/2

In [3]:
import random

def simulate_tree(p, depth):
    """
    Simulates one tree of fixed depth.
    Returns True if there exists a root-to-leaf path with at most one 1.
    """
    # Determine the root label: cost = 0 if label is 0, 1 if label is 1.
    root_cost = 0 if random.random() < p else 1
    # If the root itself has cost > 1, no valid path exists.
    if root_cost > 1:
        return False

    # current holds the cost (number of 1's encountered) along each valid path so far.
    current = [root_cost]

    # Grow the tree level by level.
    for _ in range(depth):
        next_level = []
        for cost in current:
            # For each current path, generate left and right children.
            # Left child:
            label = 0 if random.random() < p else 1
            new_cost = cost + label
            if new_cost <= 1:
                next_level.append(new_cost)
            # Right child:
            label = 0 if random.random() < p else 1
            new_cost = cost + label
            if new_cost <= 1:
                next_level.append(new_cost)
        # If no valid paths continue at this level, no valid infinite path exists.
        if not next_level:
            return False
        current = next_level

    return True

def simulate_trees(p, depth, n):
    """
    Simulates n independent trees of given depth with parameter p.
    Returns the fraction of trees that have a valid path.
    """
    successes = 0
    for _ in range(n):
        if simulate_tree(p, depth):
            successes += 1
    return successes / n

# Parameters
p = 0.5306035754    # our computed value for which the probability should be about 1/2
depth = 70          # use a large depth to approximate the infinite tree
n = 50000           # number of independent trees to simulate

# Run the simulation
estimated_probability = simulate_trees(p, depth, n)
print(f"Simulated probability with p = {p}, depth = {depth}, n = {n} trees: {estimated_probability}")

Simulated probability with p = 0.5306035754, depth = 70, n = 50000 trees: 0.50036
